# Pair Programming Scenarios


#### Setup
Set the ~~MakerSuite~~ Gemini API key with the provided helper function.

In [1]:
from utils import get_api_key
# PaLM legacy
## import google.generativeai as palm
## palm.configure(api_key=get_api_key())

# Gemini API
# From now own with Gemini API
import os
import google.generativeai as genai
from google.api_core import client_options as client_options_lib

genai.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

#### Pick the model that generates text

In [2]:
models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
model_bison

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

In [4]:
# Set the model to connect to the Gemini API
model_flash = genai.GenerativeModel(model_name='gemini-1.5-flash')

### Legacy PaLM API
#### Helper function to call the PaLM API

```Python
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)
```

### Helper function to call the Gemini API

In [5]:
def generate_text(prompt,
                  model=model_flash,
                  temperature=0.0):
    return model_flash.generate_content(prompt,
                                  generation_config={'temperature':temperature})

### Scenario 1: Improve existing code
- An LLM can help you rewrite your code in the way that's recommended for that particular language.
- You can ask an LLM to rewrite your Python code in a way that is more 'Pythonic".

In [6]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explain, in detail, what you did to improve it.
"""

In [7]:
question = """
def func_x(array)
  for i in range(len(array)):
    print(array[i])
"""

In [8]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
# Gemini API
print(completion.text)

# PaLM legacy
## print(completion.result)

The provided code iterates through a list using its index.  While functional, it's not the most Pythonic approach. Python offers more elegant and efficient ways to iterate through lists.  Here's an improved version and a detailed explanation of the changes:

```python
def func_x(array):
  for item in array:
    print(item)
```

**Improvements and Explanation:**

The original code uses `for i in range(len(array))`, which first calculates the length of the array and then iterates using the index `i` to access each element. This is less efficient and less readable than the improved version.

The improved code uses a more direct and Pythonic approach: `for item in array:`. This is called **iterating directly over the iterable**.  It directly accesses each *item* in the `array` without needing to explicitly manage indices.

**Benefits of the improved code:**

* **Readability:** The improved code is significantly more concise and easier to understand. The intent – printing each item in the a

#### Ask for multiple ways of rewriting your code

In [9]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explore multiple ways of solving the problem, and explain each.
"""

In [10]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

The provided code iterates through a list and prints each element.  While functional, it's not the most Pythonic approach. Here are several better ways, each with explanations:

**Method 1: Direct Iteration (Most Pythonic)**

```python
def func_x(array):
  for item in array:
    print(item)
```

This is the preferred method in Python.  It directly iterates over the elements of the array (list) without needing to access them by index. This is cleaner, more readable, and generally faster because it avoids the overhead of indexing.

**Method 2: List Comprehension with `print()` (Concise, but potentially less readable for beginners)**

```python
def func_x(array):
  [print(item) for item in array]
```

This uses a list comprehension, a concise way to create lists.  Here, it's used to iterate and print, but the resulting list is immediately discarded. While compact, this might be less readable for those unfamiliar with list comprehensions.  It's generally less efficient than direct iteratio

#### Paste markdown into a markdown cell

If the model outputs what looks like a table in markdown, you can copy-paste markdown into a markdown cell to make it easier to view:

For example:

| Method | Pros | Cons |
|---|---|---|
| List comprehension | Concise | Can be difficult to read for complex code |
| `enumerate()` | Easy to read | Requires an extra variable to store the index |
| `map()` | Flexible | Requires a custom function to format the output |


#### Ask the model to recommend one of the methods as most 'Pythonic'

In [11]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explore multiple ways of solving the problem, 
and tell me which is the most Pythonic
"""

In [12]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

The provided code iterates through a list and prints each element.  While functional, it's not the most Pythonic approach. Here are several better ways, with explanations of their Pythonic qualities:

**Method 1: Direct Iteration (Most Pythonic)**

```python
def func_x(array):
  for item in array:
    print(item)
```

This is the most Pythonic because it directly iterates over the *elements* of the array, rather than iterating over indices and then accessing elements by index.  It's cleaner, more readable, and avoids potential `IndexError` exceptions if you accidentally use an invalid index.

**Method 2: List Comprehension with `print()` (Less Pythonic for this specific task)**

```python
def func_x(array):
  [print(item) for item in array]
```

List comprehensions are powerful for creating new lists, but using one solely for side effects (printing) is generally considered less readable than a simple `for` loop in this case.  While it works, it's not the preferred style for this specif

### Scenario 2: Simplify code
- Ask the LLM to perform a code review.
- Note that adding/removing newline characters may affect the LLM completion that gets output by the LLM.

In [13]:
# option 1
prompt_template = """
Can you please simplify this code for a linked list in Python?

{question}

Explain in detail what you did to modify it, and why.
"""

After you try option 1, you can modify it to look like option 2 (in this markdown cell) and see how it changes the completion.
```Python
# option 2
prompt_template = """
Can you please simplify this code for a linked list in Python? \n
You are an expert in Pythonic code.

{question}

Please comment each line in detail, \n
and explain in detail what you did to modify it, and why.
"""
```

In [14]:
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.headval = None

list1 = SLinkedList()
list1.headval = Node("Mon")
e2 = Node("Tue")
e3 = Node("Wed")
list1.headval.nextval = e2
e2.nextval = e3

"""

In [15]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

The provided code is already fairly concise, but we can make it slightly more streamlined and Pythonic.  The main improvements focus on reducing redundancy and making the code easier to read and extend.

Here's a simplified version:

```python
class Node:
    def __init__(self, data):  # Simplified variable name
        self.data = data       # Simplified variable name
        self.next = None       # More concise naming

class LinkedList:  # More descriptive class name
    def __init__(self):
        self.head = None       # More concise naming

    def append(self, data):  # Added an append method for easier list creation
        new_node = Node(data)
        if not self.head:
            self.head = new_node
            return
        current = self.head
        while current.next:
            current = current.next
        current.next = new_node

# Example usage:
list1 = LinkedList()
list1.append("Mon")
list1.append("Tue")
list1.append("Wed")


#To print the list (added for demons

### Scenario 3: Write test cases

- It may help to specify that you want the LLM to output "in code" to encourage it to write unit tests instead of just returning test cases in English.

In [16]:
prompt_template = """
Can you please create test cases in code for this Python code?

{question}

Explain in detail what these test cases are designed to achieve.
"""

In [17]:
# Note that the code I'm using here was output in the previous
# section. Your output code may be different.
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0])
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head

list1 = create_linked_list(["Mon", "Tue", "Wed"])
"""

In [18]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

The provided code creates a singly linked list.  The `create_linked_list` function takes a list of data and returns the head of a newly created linked list where the elements are added to the *front* (resulting in a reversed order).  We need test cases to verify its functionality under various conditions.

Here's Python code with test cases using the `unittest` module:

```python
import unittest

class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0]) if data else None #Handle empty list case
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head

class TestCreateLinkedList(unittest.TestCase):

    def test_empty_list(self):
        head = create_linked_list([])
        self.assertIsNone(head, "Should return None for an empty list")

    def test_single_element_list(self):


### Scenario 4: Make code more efficient
- Improve runtime by potentially avoiding inefficient methods (such as ones that use recursion when not needed).

In [19]:
prompt_template = """
Can you please make this code more efficient?

{question}

Explain in detail what you changed and why.
"""

In [20]:
question = """
# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
    # Check base case
    if high >= low:
        mid = (high + low) // 2
        if arr[mid] == x:
            return mid
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
        else:
            return binary_search(arr, mid + 1, high, x)
    else:
        return -1

# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

"""

In [21]:
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.text)

The provided code implements a recursive binary search. While functional, recursion can be less efficient than an iterative approach due to function call overhead.  Here's an improved, iterative version:

```python
def binary_search_iterative(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0

    while low <= high:
        mid = (high + low) // 2  # Integer division

        # If x is greater, ignore left half
        if arr[mid] < x:
            low = mid + 1

        # If x is smaller, ignore right half
        elif arr[mid] > x:
            high = mid - 1

        # means x is present at mid
        else:
            return mid

    # If we reach here, then the element was not present
    return -1

# Test array
arr = [2, 3, 4, 10, 40]
x = 10

# Function call
result = binary_search_iterative(arr, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")
```

**Changes and Explanations:**

1. **Iterative Appro

#### Try out the LLM-generated code
- If it uses `bisect`, you may first need to `import bisect`
- Remember to check what the generated code is actually doing.  For instance, the code may work because it is calling a predefined function (such as `bisect`), even though the rest of the code is technically broken.

In [ ]:
# Paste the LLM-generated code to inspect and debug it








### Scenario 5: Debug your code

In [22]:
prompt_template = """
Can you please help me to debug this code?

{question}

Explain in detail what you found and why it was a bug.
"""

In [23]:
# I deliberately introduced a bug into this code! Let's see if the LLM can find it.
# Note -- the model can't see this comment -- but the bug is in the
# print function. There's a circumstance where nodes can be null, and trying
# to print them would give a null error.
question = """
class Node:
   def __init__(self, data):
      self.data = data
      self.next = None
      self.prev = None

class doubly_linked_list:
   def __init__(self):
      self.head = None

# Adding data elements
   def push(self, NewVal):
      NewNode = Node(NewVal)
      NewNode.next = self.head
      if self.head is not None:
         self.head.prev = NewNode
      self.head = NewNode

# Print the Doubly Linked list in order
   def listprint(self, node):
       print(node.data),
       last = node
       node = node.next

dllist = doubly_linked_list()
dllist.push(12)
dllist.push(8)
dllist.push(62)
dllist.listprint(dllist.head)

"""

Notice in this case that we are using the default temperature of `0.7` to generate the example that you're seeing in the lecture video.  
- Since a temperature > 0 encourages more randomness in the LLM output, you may want to run this code a couple times to see what it outputs.

In [24]:
completion = generate_text(
    prompt = prompt_template.format(question=question),
    temperature = 0.7
)
print(completion.text)

The bug lies in the `listprint` function.  It correctly prints the data of the first node, but then it has a logic error that prevents it from traversing the rest of the list.

**The Problem:**

The line `node = node.next` correctly moves to the next node. However, there's no loop or recursive call to continue traversing the list.  The function only prints the data of the *first* node and then stops.  It should iterate through all nodes until it reaches the end of the list (where `node` becomes `None`).


**The Solution:**

We need to add a `while` loop to iterate through the list until the end is reached. We also need to handle the case where the list is empty.


Here's the corrected code:

```python
class Node:
   def __init__(self, data):
      self.data = data
      self.next = None
      self.prev = None

class doubly_linked_list:
   def __init__(self):
      self.head = None

# Adding data elements
   def push(self, NewVal):
      NewNode = Node(NewVal)
      NewNode.next = self.

#### Reminder to check the code
You can use an  LLM to give you insights and check for blind spots, but remember to check that the generated code is doing what you want it to do.